In [5]:
import gymnasium as gym
from stable_baselines3 import DDPG
from stable_baselines3.common.vec_env import DummyVecEnv
from pettingzoo.sisl import waterworld_v4
from stable_baselines3.common.noise import NormalActionNoise
import numpy as np

# 1. 创建PettingZoo环境
env = waterworld_v4.env()


In [6]:

# 2. 将PettingZoo环境转换为Stable-Baselines3支持的VecEnv
def make_env():
    return env

# 将环境包装成VecEnv
env = DummyVecEnv([make_env])


ValueError: The environment is of type <class 'pettingzoo.utils.wrappers.order_enforcing.OrderEnforcingWrapper'>, not a Gymnasium environment. In this case, we expect OpenAI Gym to be installed and the environment to be an OpenAI Gym environment.

In [ ]:

# 3. 添加动作噪声（DDPG需要噪声来进行探索）
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

# 4. 创建DDPG模型并开始训练
model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1)
model.learn(total_timesteps=10000, log_interval=10)

# 5. 保存模型
model.save("ddpg_waterworld")

# 6. 加载训练好的模型
model = DDPG.load("ddpg_waterworld")

# 7. 测试训练好的模型
obs = env.reset()
done = False
while not done:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()